In [1]:
import torch
import numpy as np
from stable_baselines3 import TD3


import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_simple_move  import HumanMoveSimpleAction

import common.mlflow_sb3_helper as mlf
import common.tensorboard_sb3_helper as tnb
USE_MLFLOW = False

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'ReLU',
        'pi': [256, 256],
        'qf': [256, 256],
        'vf': [256, 256],
    },
    'training': {
        'iteration_count': 1,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.003,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 1024,
        'tau': 0.005,
        'gamma': 0.99,
        'verbose': 0,
        'device': device,
    },
    'options':{
        'finish_dist':100,
        'start_dist':110,
        'delta_phi':0
    },
    'validation': {
        'validate_agent_every_n_eps': 10000,
        'log_interval': 10
    },
    'evaluation': {
        'episode_count': 3
    }
}


option_var ={
    'TD3 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    'TD3 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    'TD3 60_0':{'finish_dist':60, 'start_dist':110, 'delta_phi':0 },
    #'TD3 40_0':{'finish_dist':40, 'start_dist':110, 'delta_phi':0 },
    #'TD3 20_0':{'finish_dist':20, 'start_dist':110, 'delta_phi':0 },
    #'TD3 10_0':{'finish_dist':10, 'start_dist':110, 'delta_phi':0 },
    #'TD3 10_10':{'finish_dist':10, 'start_dist':110, 'delta_phi':10 },
    #'TD3 10_20':{'finish_dist':10, 'start_dist':110, 'delta_phi':20 },
    #'TD3 10_30':{'finish_dist':10, 'start_dist':110, 'delta_phi':30 },
    #'TD3 10_40':{'finish_dist':10, 'start_dist':110, 'delta_phi':40 },
}

option_var2 ={
    'TD3 100_0':{'finish_dist':100, 'start_dist':110, 'delta_phi':0 },
    'TD3 80_0':{'finish_dist':80, 'start_dist':110, 'delta_phi':0 },
    #'TD3 80_20':{'finish_dist':80, 'start_dist':110, 'delta_phi':20 },
    'TD3 80_45':{'finish_dist':80, 'start_dist':110, 'delta_phi':45 },
    'TD3 80_90':{'finish_dist':80, 'start_dist':110, 'delta_phi':90 },
    'TD3 80_135':{'finish_dist':80, 'start_dist':110, 'delta_phi':135 },
    'TD3 80_180':{'finish_dist':80, 'start_dist':110, 'delta_phi':180 },
    'TD3 60_180':{'finish_dist':60, 'start_dist':110, 'delta_phi':180 },
    'TD3 40_180':{'finish_dist':40, 'start_dist':110, 'delta_phi':180 },
    'TD3 20_180':{'finish_dist':20, 'start_dist':110, 'delta_phi':180 },
    'TD3 10_180':{'finish_dist':10, 'start_dist':110, 'delta_phi':180 },
}

In [3]:
options=exp_params['options']
#options=None

env = HumanMoveSimpleAction(continuous=True, target_point_rand=False, options=options)
env_render = HumanMoveSimpleAction(continuous=True, target_point_rand=False, render_mode='rgb_array', options=options)


In [4]:
log_helper = {}
log_path = {}

if USE_MLFLOW:
    log_helper = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
else:
    log_helper = tnb.TensorboardHelper("",False)

In [ ]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)


In [ ]:
#!!!!! Дорасчет старого эксперимента
experiment_id = 297
exp_name = log_helper.get_experiment(experiment_id)

print(exp_name)

In [ ]:
exp_params['exp_id'] = experiment_id

b_first = True
model = None

for name, option in option_var2.items():


    print(name)

    exp_params['env_name'] = env.name()
    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))
    exp_params['options'] = option


    env.set_options(option)
    env_render.set_options(option)

    if b_first == True:
        b_first = False
        model = TD3.load('./teached/simple_move_TD3.zip', env=env, device=device)
    else:
        print(exp_name)
        model = TD3.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'




In [ ]:
exp_params['exp_id'] = experiment_id
exp_params['algorithm_name'] = 'TD3'

b_first = True
model = None

for i in range(exp_params['evaluation']['episode_count']):

    exp_params['env_name'] = env.name()
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))


    if b_first == True:
        b_first = False
        model = TD3.load('./teached/simple_move_TD3.zip', env=env, device=device)
    else:
        print(exp_name)
        model = TD3.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=f'TD3_{i+4}',
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

    log_path[i] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'




In [ ]:
log_path